In [2]:
import torch
import math
from torch import nn, autograd, stack, cat
from torch.autograd import Variable
import torch.nn.functional as F

In [4]:
class Ensembler(nn.Module):
    def __init__(self, out_features, features_extractor, in_features, views):
        super(Ensembler, self).__init__()
        self.features_extractor = features_extractor
        self.ensemble = nn.MaxPool2d(kernel_size=(views,1))
        self.linear = nn.Linear(in_features, 100)
        self.relu = nn.ReLU()
        self.final = nn.Linear(100,out_features)

    def forward(self, views):
        views_features = [self.features_extractor(view) for view in views]
        batch,filter,h,w = views_features[0].shape
        outs = [ self.relu(self.linear(el.view(batch,-1))) for el in views_features ]
        outs = [ self.final(out) for out in outs]
        stacked_outs = torch.stack(outs,dim=1)
        out = self.ensemble(stacked_outs)
        return out.squeeze()